<a href="https://colab.research.google.com/github/ZabalaGaston/HPC/blob/main/Zabala_Gaston_ejercicio_3_Reentrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1 Introducción

La ley de la gravitación universal formulada por Isaac Newton postula que la fuerza que ejerce una partícula puntual con masa **m1** sobre otra con masa **m2** es directamente proporcional al producto de las masas, e inversamente proporcional al cuadrado de la distancia que las separa: [1]

>$F = G\frac{m1* m2}{r^2} $

**G** es la constante de gravitación universal y vale aproximadamente 6,674 * 10^-11

La masa de la Tierra es 5,974 × 1024 kg. 

A modo de demostración del ejercicio, se incluyo al inicio el calculo en el lenugaje Python[1,2,3] con el fin de representar en forma gráfica el resultado obtenido. 

En este ejercico, se busca realizar una comparación directa entre una ejecución en serie y una en paralelo sobre el mismo juego de valores. Para ello, se utilizara **OpenMP[5]** para realizar en varios hilos de ejecución una programación paralela.



#2 Armado del ambiente

#3 Desarrollo
Ejemplo desarrollado en Python a modo de demostración grafico.

In [98]:
#@title ## Parámetros de ejecución

#@markdown ---
#@markdown ### Especifique la cantidad de elementos a las cuales se les va a calcular la fuerza gravitatoria:
elementos = 2#@param {type:"number"}

try:
  import numpy as np
  import matplotlib.pyplot as plt
  from matplotlib import animation, rc
  from IPython.display import HTML

  colores = ['green'] * elementos

  distancia = 6378140
  G = 6.674 * pow(10,-11);
  masaTierra = 5.974 * pow(10,24);

  v_masas  = np.random.uniform(0, 100, size=elementos)
  v_fg  = np.array(elementos)  

  for i in range( 0, elementos ):
    v_fg[elementos] = g * masaTierra  *  masaCuerpos[i] / pow(DISTANCIA,2);



  # Representacion del resultado. ----------------------------------------------
#  fig, ax = plt.subplots()
#  ax1 = fig.add_subplot(projection="3d")
#  ax1.scatter(x_cpu, y_cpu, z_cpu, c=colores, s=m_cpu)
#  ax1.scatter(cdmx, cdmy, cdmz, c='red', s=10)
except ModuleNotFoundError:
  print("Error en modulos utilizados")
except Exception as e:
  print("Error:",e.args)

##3.1 Generar code_fg.cpp
Mediante código Python, genero una cadena de texto con código C. Luego, guardo la cadena en el archivo code_fg.cpp

In [89]:
code = """

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <math.h>
#include <omp.h> //Cabecera OpenMP

//------------------------------------------------
// Macros para medir el tiempo

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL     1
#define HTH_FG_SEC  2
#define HTH_FG_OMP  3
#define DISTANCIA  6378140
//---------------------------------------------------------

int main(int argc, char*argv[])
{
  int i,c;
  TIEMPO_INI ( HTH_TOTAL )

  //Leo los parametros.
  if (argc != 3 )
  {
    std::cerr<<"Error al ingresar los parámetros" <<std::endl;
    exit( - 1 );
   }

   int cantidadCuerpos  =  atoi( argv[1] );
   int ciclos           =  atoi( argv[2] );

   double masaTierra;
   double g;

   g = 6.674 * pow(10,-11);
   masaTierra = 5.974 * pow(10,24);
    
   if ( cantidadCuerpos <= 0)
   {
      std::cerr<<"La cantidad de cuerpos debe ser mayor a 0 [Parametro 1]" <<std::endl;
      exit( - 2 );
   }

   if ( ciclos <= 0)
   {
      std::cerr<<"La cantidad de ciclos debe ser mayor a 0 [Parametro 2]" <<std::endl;
      exit( - 3 );
   }

   //-----------------------------------------------------
   // Defino la memoria de los vectores para la masa de los cuerpos

   std:: vector<double>   masaCuerpos ( cantidadCuerpos );
   std:: vector<double>   fuerzaGrav ( cantidadCuerpos );

   for (int i = 0; i < cantidadCuerpos; i++)
   {
      masaCuerpos[i] = (rand()/(double)RAND_MAX)*100.0; 
   }

  //------------------------------------------
  // Realizo el cálculo de la fuerza gravitatoria de los cuerpos con relación a la tierra en forma secuencial

  TIEMPO_INI ( HTH_FG_SEC )

  for(c=0; c<ciclos; c++)
  {
    for(i=0; i< cantidadCuerpos ; i++)
    {
       fuerzaGrav[i] = g * masaTierra  *  masaCuerpos[i] / pow(DISTANCIA,2);
    }
  }

  TIEMPO_FIN ( HTH_FG_SEC )

  // ---------------------------------------------
  // Realizo el cálculo de la aceleración con OpenMP

  TIEMPO_INI ( HTH_FG_OMP )

  for(c=0; c<ciclos; c++)
  {
    #pragma omp parallel for
    for(i=0; i< cantidadCuerpos ; i++)
    {
      fuerzaGrav[i] = g * masaTierra  *  masaCuerpos[i] / pow(DISTANCIA,2);
    }
  }

  TIEMPO_FIN ( HTH_FG_OMP )

  //------------------------------------------------
  std::cout<<"Valores Reales  : " <<std::endl;
  std::cout<<"Tiempo Fuerza gravitatoria sec : " <<TIEMPO_GET(HTH_FG_SEC)<< " [ms]"<<std::endl;
  std::cout<<"Tiempo Fuerza gravitatoria Omp : " <<TIEMPO_GET(HTH_FG_OMP)<< " [ms]"<<std::endl;
  std::cout<<std::endl;  
  std::cout<<"SpeedUp          :  (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_FG_SEC)<<" / "<<TIEMPO_GET(HTH_FG_OMP)<<" = "<<TIEMPO_GET(HTH_FG_SEC)/TIEMPO_GET(HTH_FG_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_FG_SEC)/TIEMPO_GET(HTH_FG_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_FG_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_FG_OMP))<<std::endl;

  std::cout<<"-------------------------------------------------------------------------------------"<<std::endl;
  std::cout<<std::endl;
  std::cout<<"Valores Ideal: "<<std::endl;
  TIEMPO_GET(HTH_FG_OMP) = TIEMPO_GET(HTH_FG_SEC) / 2;
  std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_FG_SEC)<<" / "<<TIEMPO_GET(HTH_FG_OMP)<<" = "<<TIEMPO_GET(HTH_FG_SEC)/TIEMPO_GET(HTH_FG_OMP)<<std::endl;
  std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_FG_SEC)/TIEMPO_GET(HTH_FG_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_FG_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_FG_OMP))<<std::endl;
  std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_FG_SEC)<<" = "<<TIEMPO_GET(HTH_FG_SEC)<<std::endl;
  std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_FG_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_FG_OMP)<<std::endl;
  std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_FG_OMP)<<" - "<<TIEMPO_GET(HTH_FG_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_FG_OMP))-TIEMPO_GET(HTH_FG_SEC)<<std::endl;

}
"""

text_file = open("code_fg.cpp","w")
text_file.write(code)
text_file.close()



##3.2 Compilación de código C (cálculo de aceleración)

In [90]:
!g++ -o fg -fopenmp code_fg.cpp

##3.3 Configuro cantidad de hilos a ejecutar en forma paralela

In [ ]:
%env OMP_NUM_THREADS=2

Ejecución del calculo de la fuerza gravitatoria en Serie y Paralela

In [96]:
# --------------------------------------------
from datetime import datetime
tiempo_total = datetime.now()

# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

#----------------------------------------------------------------------
try:

#---- Ejecución

# Parámetros:
# 2 -> Cantidad de Cuerpos
# 3 -> Cantidad de ciclos

  !./fg 10000000 2
#----

  tiempo_total = datetime.now() - tiempo_total;
  print("Tiempo Total: ", tiempo_en_ms( tiempo_total ), "[ms]" )

except Exception as e:
  print("Error:",e.args)

Valores Reales  : 
Tiempo Fuerza gravitatoria sec : 332.5 [ms]
Tiempo Fuerza gravitatoria Omp : 312.569 [ms]

SpeedUp          :  (tiempo Secuencial/tiempo paralelo) : 332.5 / 312.569 = 1.06377
Eficiencia       : SpeedUp/nro procesadores            : 1.06377 / 2 = 0.531883
-------------------------------------------------------------------------------------

Valores Ideal: 
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 332.5 / 166.25 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo             : 1 * 332.5 = 332.5
Coste Omp        : nro procesadores*Tiempo             : 2 * 166.25 = 332.5
Funcion Overhead : Coste Omp - tiempo Secuencial       : 332.5 - 332.5 = 0
Tiempo Total:  1015.106 [ms]


#4 Tabla de pasos
Solo aplica a la comparacion entre Serie - Paralelo con OpenMP


 Procesador | Función | Detalle
------------|---------|----------
CPU      |  #includes | Librerias necsarias para el desarrollo del ejercicion en C++.
CPU      |  #define     | Definicion de macros utilizadas en el proceso..
CPU      |main()       | 	Funcion principal.
CPU      |  TIEMPO_INI  | Toma el tiempo inicial para la ejecucion en serie..
CPU      |  argc != 3        | Validacion de parametro.
CPU      |  std::vector|Definición de vector 
CPU      |  for(...)        | Cálculo de la fuerza de gravitatoria
CPU      |  TIEMPO_FIN               | Captura el tiempo final para la ejecución en serie.
CPU      | TIEMPO_INI                | Toma el tiempo inicial para la ejecución en paralelo.
CPU      | #pragma | Defino el sector que se va a ejecutar en paralelo, indicando que variables van a ser compartidas por los distintos hilos de ejecucion
CPU | for(...) | Ejecución de cálculo de fuerza gravitatoria.
CPU | TIEMPO_FIN | Toma el tiempo final para la ejecución en paralelo
CPU | std:: | Muestro las estadísticas
CPU | open(...) | Apertura del archivo C++.
CPU | write(...) | Escritura del archivo C++.
CPU | close (...) | Cierre del archivo C++.
CPU | !g++ | Compilación del código.
CPU | %env | Definición de cantidad hijos a ejecutar en paralelo.
CPU | !./fg 10000000 50 | Ejecución del código, pasando por parámetro la cantidad de cuerpos y la cantidad de ciclos 


#5 Conclusion

Mediante la realización de ejercicio, pude comprender el funcionamiento con hilos en ejecución a traves de OpenMP.

Podemos observar que los tiempos para calcular la fuerza gravitatoria de manera  secuencial son mayores que al calcular la misma en paralelo (OpenMP).

*Tiempo Acel sec : 828.774 [ms] <br>
Tiempo Acel Omp : 763.55 [ms] <br>
Tiempo Total:     1729.769 [ms]*


En los casos cuando la cantidad de elementos para calcularle al fuerza gravitatoria es pequeño, la ejecución en serie obtiene mejores resultados debido a que se agrega un tiempo extra en comenzar a ejecutar las operaciones en paralelo. En cambio, cuando la cantidad de elementos es elevada, en paralelo se ejecuta mas rapido, gracias a que cada hilo resuelve parte del problema y todo se ejecuta en fomra simultanea.

El punto más relevante en este ejercicio es demostrar el funcionamiento y utilizacion de OpenMP para la ejecución en paralelo. La sentencia #pragma omp parallel for, nos indica que la parte de código que la comprende puede ser ejecutada por varios hilos. Se crea una tarea implícita para cada hilo perteneciente al equipo de hilos creado por el parallel.(Por defecto, openmp sabe que vamos a iterar por el valor i).  

En cuanto a lecciones aprendidas, comprendí el funcionamiento de OpenMP. <br>
Como mejora, se podrían agregar métricas de ejecución para un posterior análisis.

#6 Bibliografía

[1] Aceleracion: [Referencia](https://www.fisicalab.com/apartado/aceleracion)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb) 

[3] Numpy: [Referencia](https://numpy.org/doc/1.16/reference/routines.random.html)

[4] Sliders Parametros: [Referencia](https://colab.research.google.com/notebooks/forms.ipynb)

[5] openMP: [Enlace](https://es.wikipedia.org/wiki/OpenMP)